In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark import SparkContext

sc = SparkContext()

# edges data
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]
edges_rdd = sc.parallelize(edges)

# inicializar el nodo de inicio con costo 0
source_node = 1
vertices = edges_rdd.flatMap(lambda x: x[:2]).distinct()
initial_costs = vertices.map(lambda v: (v, 0) if v == source_node else (v, float('inf')))

# converte la lista de bordes en un RDD
edges_rdd = edges_rdd.map(lambda x: (x[0], (x[1], x[2])))

# bucle hasta que los costos no cambien
while True:
    # Paso 2: enviar costos acumulados a vecinos
    new_costs = initial_costs.join(edges_rdd).map(lambda x: (x[1][1][0], x[1][0] + x[1][1][1]))

    # Paso 3: reducir al costo mínimo
    new_costs = new_costs.reduceByKey(min)

    # verificación de convergencia
    if new_costs.join(initial_costs).map(lambda x: abs(x[1][0] - x[1][1]) < 0.01).reduce(lambda a, b: a and b):
        break
    else:
        initial_costs = new_costs

# imprime los resultados
print(initial_costs.collect())

In [ ]:
# Definimos la función para actualizar las distancias mínimas
def actualizar_distancia(distancias, vertice, peso):
    if vertice not in distancias or peso < distancias[vertice]:
        distancias[vertice] = peso

# Lista de aristas
edges = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1)]

# Creamos un diccionario para almacenar las distancias mínimas
distancias = {1: 0}

# Creamos un diccionario para almacenar los nodos visitados
visitados = {}

# Iteramos hasta que hayamos visitado todos los nodos
while len(visitados) < len(distancias):
    # Encontramos el nodo no visitado con la distancia mínima
    nodo_actual = min(distancias, key=lambda x: distancias[x] if x not in visitados else float('inf'))
    visitados[nodo_actual] = distancias[nodo_actual]

    # Actualizamos las distancias mínimas de los vecinos del nodo actual
    for u, v, peso in edges:
        if u == nodo_actual:
            actualizar_distancia(distancias, v, distancias[u] + peso)

# Imprimimos los resultados
for nodo in distancias:
    distancia_minima = distancias[nodo]
    print(f"Distancia mínima desde el nodo 1 hasta el nodo {nodo}: {distancia_minima}")